In [1]:
root_dir = '/home2/glee/dissertation/1_tech_gen_impact/class2class/Tech_Gen/'
master_dir = '/home2/glee/dissertation/1_tech_gen_impact/master/Tech_Gen/'
import sys
sys.path.append(root_dir)

import copy
import gc
import os
import argparse
import math
import time
import pickle
import re
import multiprocess as mp
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
sys.path.append("/share/tml_package")
from tml import utils
from scipy import io
from tqdm import tqdm
from collections import OrderedDict

import torch
from torch.nn import functional as F
from torch.nn import DataParallel as DP
from torch.utils.data import TensorDataset, DataLoader, Subset, Dataset
from accelerate import Accelerator
import pytorch_model_summary

import optuna
from optuna.samplers import RandomSampler, TPESampler
from optuna.integration import SkoptSampler

import numpy as np
import pandas as pd
import scipy.stats
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import matthews_corrcoef, precision_recall_fscore_support, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

from data import TechDataset, CVSampler
from models import Transformer, Predictor
from train_utils import EarlyStopping, perf_eval, objective_cv, build_model, train_model, validate_model_mp
from utils import token2class, DotDict, to_device

from cleantext.sklearn import CleanTransformer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# PART 1: Configuration

In [9]:
args = argparse.Namespace(
    data_type="class+claim",
    data_file = "collection_[H01L,H10][2017].csv",
    target_ipc="H01L",
    pred_type="classification",
    n_TC = 5,
    use_pretrained_tokenizer=False,
    do_train=None,
    do_tune=None,
    n_folds=None,
    batch_size=16,
    max_epochs=50,
    use_accelerator=None,
    do_save=False,
    n_gpus=4,
    light=True,
#     config_file=os.path.join(root_dir, "configs", "USED_configs", "[CONFIGS]2023-04-11_00:39.json"),
    config_file=None,
    eval_train_set=False)

data_dir = os.path.join(master_dir, "data")
model_dir = os.path.join(root_dir, "models")
result_dir = os.path.join(root_dir, "results")
config_dir = os.path.join(root_dir, "configs")

# args = parser.parse_args()
if args.config_file is not None:
    config_file = args.config_file
else:
    config_file = os.path.join(config_dir, "configs_light.json") if args.light else os.path.join(config_dir, "configs.json")
configs = DotDict().load(config_file)
org_config_keys = {key: list(configs[key].keys()) for key in configs.keys()}

instant_configs = {key: value for (key, value) in vars(args).items() if value is not None} # if any argument passed when main.py executed
instant_configs_for_update = {configkey: {key: value for (key,value) in instant_configs.items() if key in org_config_keys[configkey]} for configkey in org_config_keys.keys()}
for key, value in configs.items():
    value.update(instant_configs_for_update[key])

regex_ipc = re.compile('[A-Z](?![\\D])')
if regex_ipc.match(configs.data.target_ipc) is None:
    configs.data.update({"target_ipc": "ALL"})
elif len(configs.data.target_ipc) > 5:
    configs.data.update({"target_ipc": configs.data.target_ipc[:4]})

if configs.model.model_type == "enc-pred-dec":
    configs.train.loss_weights["recon"] = configs.train.loss_weights["recon"] / sum(configs.train.loss_weights.values())
    configs.train.loss_weights["y"] = 1 - configs.train.loss_weights["recon"]
elif configs.model.model_type == "enc-pred":
    configs.train.loss_weights = {"recon": 0, "y": 1}
elif configs.model.model_type == "enc-dec":
    configs.train.loss_weights = {"recon": 1, "y": 0}

if configs.train.use_accelerator:
    accelerator = Accelerator()
    device_ids = list(range(torch.cuda.device_count()))
    device = accelerator.device
    configs.train.update({"accelerator": accelerator})
else:
    if torch.cuda.is_available():
        device_ids = list(range(torch.cuda.device_count()))
        gpu_usages = [np.sum([float(usage.split("uses")[-1].replace(" ","").replace("MB","")) for usage in torch.cuda.list_gpu_processes(id).split("GPU memory") if not usage=="" and "no processes are running" not in usage]) for id in device_ids]
        device_ids = np.argsort(gpu_usages)[:configs.train.n_gpus]
        device_ids = list(map(lambda x: torch.device('cuda', x),list(device_ids)))
        device = device_ids[0] # main device
        torch.cuda.set_device(device)
    else:
        device = torch.device('cpu')
        device_ids = []

configs.data.update({"root_dir": root_dir,
                        "data_dir": data_dir,
                        "model_dir": model_dir,
                        "result_dir": result_dir,
                        "pretrained_enc": configs.model.pretrained_enc,
                        "pretrained_dec": configs.model.pretrained_dec})
configs.train.update({"device": device,
                        "device_ids": device_ids,
                        "root_dir": root_dir,
                        "data_dir": data_dir,
                        "model_dir": model_dir,
                        "use_keywords": configs.data.use_keywords,
                        "early_stop_patience": int(0.3*configs.train.max_epochs)})
configs.model.update({"device": device,
                        "device_ids": device_ids,
                        "n_directions": 2 if configs.model.bidirec else 1,
                        "use_accelerator": configs.train.use_accelerator})

## Set hyperparameters for model training (To be TUNED)
if configs.train.do_train and configs.train.do_tune:
    n_layers = configs.model.n_layers = None
    d_embedding = configs.model.d_embedding = None
    d_enc_hidden = configs.model.d_enc_hidden = None
    d_pred_hidden = configs.model.d_pred_hidden = None
    d_latent = 64
    learning_rate = configs.train.learning_rate = None
    batch_size = configs.train.batch_size = None
    config_name = "HPARAM_TUNING"
    final_model_path = None
else:
    n_layers = configs.model.n_layers
    d_embedding = configs.model.d_embedding
    d_enc_hidden = configs.model.d_enc_hidden
    d_pred_hidden = configs.model.d_pred_hidden
    d_latent = configs.model.d_enc_hidden * configs.model.n_directions
    d_latent = 64

    configs.model.update({"d_latent": d_latent})

    key_components = {"data": ["target_ipc", "vocab_size"], "model": ["n_layers", "d_enc_hidden", "d_pred_hidden", "d_latent", "d_embedding", "d_ff", "n_head", "d_head"], "train": ["learning_rate", "batch_size", "max_epochs"]}
    config_name = ""
    for key in key_components.keys():
        for component in key_components[key]:
            config_name += "["+str(configs[key][component])+component+"]"
    final_model_path = os.path.join(model_dir, f"[Final_model]{config_name}.ckpt")

configs.train.update({"config_name": config_name,
                        "final_model_path": final_model_path})

# PART 2: Dataset setting

In [16]:
tstart = time.time()
org_config_keys_temp = copy.copy(org_config_keys["data"])
org_config_keys_temp.pop(org_config_keys_temp.index("use_pretrained_tokenizer"))
org_config_keys_temp.pop(org_config_keys_temp.index("data_file"))
dataset_config_name = "-".join([str(key)+"="+str(value) for (key,value) in configs.data.items() if key in org_config_keys_temp])
dataset_path = os.path.join(data_dir, "pickled_dataset", "[tech_dataset]"+dataset_config_name+".pickle")
if os.path.exists(dataset_path) and args.do_save is False:
    print("Load pickled dataset...")
    with open(dataset_path, "rb") as f:
        tech_dataset = pickle.load(f)   # Load pickled dataset if dataset with same configuration already saved
        if tech_dataset.pretrained_enc != configs.data.pretrained_enc or tech_dataset.pretrained_dec != configs.data.pretrained_dec:
                tech_dataset.pretrained_enc = configs.data.pretrained_enc
                tech_dataset.pretrained_dec = configs.data.pretrained_dec
                tech_dataset.tokenizers = tech_dataset.get_tokenizers()
        for tk in tech_dataset.tokenizers.values():
            if "vocab_size" not in dir(tk):
                tk.vocab_size = tk.get_vocab_size()
    print("Pickled dataset loaded")
else:
    print("Make dataset...")
    tech_dataset = TechDataset(configs.data)
    with open(dataset_path, "wb") as f:
        tech_dataset.rawdata = None
        pickle.dump(tech_dataset, f)
tend = time.time()
print(f"{np.round(tend-tstart,4)} sec elapsed for loading patents for class [{configs.data.target_ipc}]")

Load pickled dataset...
Pickled dataset loaded
0.5718 sec elapsed for loading patents for class [H01L]


In [17]:
configs.model.update({"tokenizers": tech_dataset.tokenizers,
                    "n_enc_seq_claim": tech_dataset.max_seq_len_claim,
                    "n_dec_seq_claim": tech_dataset.max_seq_len_claim,
                    "n_enc_seq_class": tech_dataset.max_seq_len_class,
                    "n_dec_seq_class": tech_dataset.max_seq_len_class,
                    "n_outputs": 1 if configs.data.pred_type=="regression" else tech_dataset.n_outputs,
                    "i_padding": tech_dataset.tokenizers["class_enc"].token_to_id("<PAD>")})

# PART 3: Training

In [18]:
sampler = CVSampler(tech_dataset, n_folds=configs.train.n_folds, test_ratio=0.1, stratify=True)
cv_idx = sampler.get_idx_dict()
print(f"#Samples\nTrain: {len(cv_idx[0]['train'])}, Validation: {len(cv_idx[0]['val'])}, Test: {len(cv_idx[0]['test'])}")

#Samples
Train: 24389, Validation: 6098, Test: 3388


## PART 3-2: Dataset construction and model training

In [19]:
# import importlib
# import models, train_utils, parallel, utils
# importlib.reload(models)
# importlib.reload(train_utils)
# importlib.reload(parallel)
# importlib.reload(utils)
# from train_utils import build_model
# from models import CLS2CLS
# from utils import to_device

In [20]:
## Construct datasets
train_idx = cv_idx[0]['train']
val_idx = cv_idx[0]['val']
test_idx = cv_idx[0]['test']
whole_idx = np.concatenate([train_idx, val_idx])

train_dataset = Subset(tech_dataset, train_idx)
val_dataset = Subset(tech_dataset, val_idx)
test_dataset = Subset(tech_dataset, test_idx)
whole_dataset = Subset(tech_dataset, whole_idx)

train_loader = DataLoader(train_dataset, batch_size=configs.train.batch_size, shuffle=True, num_workers=4, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=configs.train.batch_size if len(val_idx)>configs.train.batch_size else len(val_idx), shuffle=True, num_workers=4, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=configs.train.batch_size if len(test_idx)>configs.train.batch_size else len(test_idx), shuffle=False, num_workers=4)
whole_loader = DataLoader(whole_dataset, batch_size=configs.train.batch_size, shuffle=False, num_workers=4)

## Load best model or build model
final_model = build_model(configs.model, tokenizers=tech_dataset.tokenizers)

In [21]:
batch_data = next(iter(train_loader))
batch_data = to_device(batch_data, device)

In [22]:
outputs = final_model.module(batch_data["text_inputs"], batch_data["text_outputs"])

In [27]:
outputs["dec_outputs"].argmax(2)

tensor([[    0,  4930,  2868,  ...,  2086, 13263, 10467],
        [    0, 11899,  5099,  ..., 10532,  6920, 10671],
        [    0, 13939, 14257,  ..., 14522,  4805,  3692],
        ...,
        [    0,  8819, 11417,  ..., 12581,  2584, 10467],
        [    0, 12789, 12789,  ..., 10582, 10467,  4910],
        [    0, 12125,  2888,  ...,  8100,  9805,  6568]], device='cuda:0')

In [32]:
next_input.view(-1)

torch.Size([16])

In [34]:
enc_outputs, z, mu, logvar = final_model.module.encode(batch_data["text_inputs"])

In [36]:
batch_size = configs.train.batch_size
next_input = torch.tensor(np.tile([final_model.module.tokenizers["class_enc"].vocab_w2i["<SOS>"]], batch_size), device=device)
next_hidden = z

In [57]:
output, next_hidden, pred_token = final_model.module.pred_next(next_input, next_hidden, enc_outputs["class"])

In [65]:
dec_outputs = torch.zeros((batch_size, final_model.module.config.n_dec_seq_class, final_model.module.tokenizers["class_dec"].vocab_size), device=device)
# dec_outputs += final_model.module.tokenizers["class_dec"].vocab_w2i["<PAD>"]

In [41]:
pred_token

tensor([11421,  5115,  6975,  7191,  2979,  9106,  4748, 11335,  9719,  2326,
        14016,  1577,  4748,  3497, 11899,  4809], device='cuda:0')

In [69]:
dec_outputs.argmax(2).shape

torch.Size([16, 100])

In [42]:
output, hidden = final_model.module.decoder(next_input, next_hidden, enc_outputs["class"])

In [64]:
output.shape

torch.Size([16, 14552])

In [56]:
next_input

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [51]:
final_model.module.tokenizers["class_dec"].vocab_w2i["<EOS>"]

2

In [53]:
pred_token = output.argmax(1)
pred_token

tensor([ 8172, 13575, 13624, 10551,  6114, 14059, 13434, 13575, 14016, 11421,
        13434,  3895, 13773,  9093, 12546, 11253], device='cuda:0')

In [70]:
pred_token[next_input.view(-1)==final_model.module.tokenizers["class_dec"].vocab_w2i["<EOS>"]] = 3

In [80]:
tech_dataset.tokenizers["class_enc"].vocab_w2i["<PAD>"]

1

In [78]:
pred_token

tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], device='cuda:0')

In [77]:
pred_token[next_input.view(-1)==0] = 3

In [76]:
next_input.view(-1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [74]:
pred_token

tensor([13434,  2704, 13434,  8631,  8631, 10006, 13434, 11421, 11421,  7754,
        14016, 13434, 10006,  7157,  3446, 12536], device='cuda:0')

In [47]:
next_input.view(-1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [28]:
batch_data["text_outputs"]

tensor([[    0,  9343,  9331,  ...,     1,     1,     1],
        [    0, 12027, 12007,  ...,     1,     1,     1],
        [    0, 10656,  9732,  ...,     1,     1,     1],
        ...,
        [    0, 14170, 10377,  ...,     1,     1,     1],
        [    0, 11956,  9072,  ...,     1,     1,     1],
        [    0, 11781, 11694,  ...,     1,     1,     1]], device='cuda:0')

In [26]:
configs.model.n_dec_seq_class

100

## PART 3-3: Training evaluation

In [ ]:
if args.eval_train_set:
    ## Evaluation on train dataset
    print("Validate model on train dataset")
    # trues_recon_train, preds_recon_train, trues_y_train, preds_y_train = validate_model(final_model, whole_loader, configs.model, configs.train)
    val_res_train = validate_model_mp(final_model, whole_dataset, mp=mp, model_params=configs.model, train_params=configs.train)
    trues_recon_train = np.concatenate([res["recon"]["true"] for res in val_res_train.values()])
    preds_recon_train = np.concatenate([res["recon"]["pred"] for res in val_res_train.values()])
    trues_y_train = np.concatenate([res["y"]["true"] for res in val_res_train.values()])
    preds_y_train = np.concatenate([res["y"]["pred"] for res in val_res_train.values()])

    eval_recon_train = perf_eval("TRAIN_SET", trues_recon_train, preds_recon_train, configs=configs, pred_type='generative', tokenizer=final_model.module.tokenizer)
    eval_recon_train = perf_eval("TRAIN_SET", trues_recon_train, preds_recon_train, configs=configs, pred_type='generative', tokenizer=final_model.module.tokenizer)
    eval_y_train = perf_eval("TRAIN_SET", trues_y_train, preds_y_train, configs=configs, pred_type=configs.data.pred_type)
    if configs.data.pred_type == "classification":
        eval_y_train, confmat_y_train = eval_y_train
else:
    eval_recon_train = eval_y_train = None

## Evaluation on test dataset
print("Validate model on test dataset")
# trues_recon_test, preds_recon_test, trues_y_test, preds_y_test = validate_model(final_model, test_loader, configs.model, configs.train)
val_res_test = validate_model_mp(final_model, test_dataset, mp=mp, batch_size=64, model_params=configs.model, train_params=configs.train)
trues_recon_test = np.concatenate([res["recon"]["true"] for res in val_res_test.values()])
preds_recon_test = np.concatenate([res["recon"]["pred"] for res in val_res_test.values()])
trues_y_test = np.concatenate([res["y"]["true"] for res in val_res_test.values()])
preds_y_test = np.concatenate([res["y"]["pred"] for res in val_res_test.values()])

eval_recon_test = perf_eval("TEST_SET", trues_recon_test, preds_recon_test, configs=configs,  pred_type='generative', tokenizer=final_model.module.tokenizer)
eval_y_test = perf_eval("TEST_SET", trues_y_test, preds_y_test, configs=configs, pred_type=configs.data.pred_type)
if configs.data.pred_type == "classification":
    eval_y_test, confmat_y_test = eval_y_test

eval_recon_res = pd.concat([eval_recon_train, eval_recon_test], axis=0)
eval_y_res = pd.concat([eval_y_train, eval_y_test], axis=0)
if configs.data.pred_type == "classification":
    confmat_y_res = pd.concat([confmat_y_train, confmat_y_test], axis=0)
